# Content & Image Trawling 
## <i> Using Spruce </i> 
## version 1.0 

In [1]:
sites = {"TomsHW":"http://Tomshardware.Com", "CNET":"http://cnet.com","Verge":"http://TheVerge.com"}

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def toms_search(search_term): 
    url = "http://TomsHardware.com"

    #Build out toms hardwar URL 
    #   Example for washing machines: https://www.tomshardware.com/search?searchTerm=Washing+Machine

    search_term = search_term.replace(" ", "+")
    search_term = url + "/search?searchTerm=" + search_term 
    return search_term 

def spruce_search_bestlinks(search_term): 
    ## Example: https://www.thespruce.com/search?q=Washing%20Machine&offset=24
    url = 'https://www.thespruce.com/'
    search_term = search_term.replace(" ", "+")
    search_url = url + "/search?q=" + search_term 

    #Pull out the search results 
    src = requests.get(search_url).text 
    soup = BeautifulSoup(src, 'html.parser')

    link_list = []
    for item in  soup.find_all("div", {"class": "comp search-layout-container mntl-block"}):
        for link in item.find_all('a'):
            link_list.append(link.get('href')) 
    
    #"the 'best' reviews will start with /best-
    search_list = []
    for link in link_list:
        if ('/best-' in str(link).lower()): search_list.append(link)


    return search_list  
    #Look for something like "The XXX Best <Search Term>s of 2020"
    #   This will return results in 24 blocks
    #   Next page is like https://www.thespruce.com/search?q=Washing%20Machine&offset=24 

def spruce_search_top_pics(search_url_spruce):
    src = requests.get(search_url_spruce).text 
    soup = BeautifulSoup(src, 'html.parser')
    #The "Our Top Pics are in the class: 'comp mntl-sc-block-callout-body mntl-text-block'"
    item_cat=[]
    item_name=[]
    item_source=[]
    item_link=[]
    

    #Print out the title 
    try:
        print (soup.find_all('h1')[0].text)
    except: 
        print ("No Title Found")

    for item in  soup.find_all("div", {"class": "comp mntl-sc-block-callout-body mntl-text-block"}):
        #The category is in the "Strong"
        #print ("Looking for the top categoies")
        try: 
            for strong in item.find_all('strong'): 
                #"this is the "Best for" category
                item_str = str(strong.contents[0])
                #So these cateogies tend to start with "Best"
                if 'best' in item_str.lower():
                    item_str   = item_str.replace(":","")
                    item_cat.append(item_str)
        except: print("No Strong contents found")
        #Look for the affilate link
        #print("Looking for links")
        try: 
            for link in (item.find_all('a', href=True, attrs={'data-source':'affiliate'}) + 
            item.find_all('a', href=True, attrs={'data-source':'inlineLink'})):
                        #print('------------')
                        #print(link)
                        #print(link.contents[0])
                        #Add the name, remove 'at Amazon' if it is there
                        item_name.append(str(link.contents[0]).replace("at Amazon", ""))
                        #print(link['href'])
                        #Add their affilate link
                        item_link.append(link['href'])
                        #Att the source 
                        item_source.append(link['data-component'])
        except: print("Nothing found")
    #Only get as many cats as they are 
    cat_count = len(item_cat)
    data = {'Name':item_name[0:cat_count-1],
            'Cat':item_cat[0:cat_count-1],
            'Source':item_source[0:cat_count-1],
            'Link':item_link[0:cat_count-1]} 
    #df = pd.DataFrame(data)
    return pd.DataFrame(data)
   # return data

def get_table_spruce(item_name): 
    try:
        top_link = spruce_search_bestlinks(item_name)[0]
        df = spruce_search_top_pics(top_link)
    except: 
        print ("Didn't find anything")
        return  
    return df

In [3]:
#For example. Lets look for the best doorbell cams 
search_list = spruce_search_bestlinks('doorbell camera')
search_list 

['https://www.thespruce.com/best-doorbell-cameras-4147384',
 'https://www.thespruce.com/best-home-security-companies-4692680',
 'https://www.thespruce.com/best-wireless-doorbells-4178455',
 'https://www.thespruce.com/best-diy-home-security-systems-4143582',
 'https://www.thespruce.com/best-science-toys-4150533']

In [4]:
spruce_search_top_pics('https://www.thespruce.com/best-trivia-games-4164628')

The 7 Best Trivia Games of 2020


,Name,Cat,Source,Link
0,Smart Ass,Best for Groups,amazon,https://www.amazon.com/University-Games-1360-S...
1,Trivial Pursuit,Best Classic,amazon,https://www.amazon.com/Trivial-Pursuit-Game-Cl...
2,Professor Noggin's Card Games,Best for Kids,amazon,https://www.amazon.com/Professor-Noggins-Histo...
3,Best of TV & Movies,Best TV and Movie Trivia,amazon,https://www.amazon.com/Spin-Master-Games-Movie...
4,iKNOW Innovative Trivia Game,Best Strategy,amazon,https://www.amazon.com/Tactic-Games-US-52658-I...


In [5]:
diction

NameError: name 'diction' is not defined

In [6]:
get_table_spruce("doorbell camera")

The 8 Best Doorbell Cameras of 2020


,Name,Cat,Source,Link
0,Ring Video Doorbell 3,Best Overall,amazon,https://www.amazon.com/dp/B0849J7W5X/?tag=thes...
1,Ring Video Doorbell Pro at Home Depot,Best WiFi,link,https://www.homedepot.com/p/Ring-1080P-HD-Wi-F...
2,Google Nest Hello Smart WiFi Video Doorbell at...,Best Wireless,link,https://www.homedepot.com/p/Google-Nest-Hello-...
3,Arlo Video Doorbell at Target,Best Motion Detection,link,https://www.target.com/p/arlo-video-doorbell/-...
4,SimpliSafe Video Doorbell Pro at Best Buy,Best with Security,link,https://www.bestbuy.com/site/simplisafe-pro-vi...
5,Eufy Security Video Doorbell at Eufy Life,Best Privacy,link,https://www.eufylife.com/products/variant/vide...
6,RemoBell S Smart Doorbell Camera at Home Depot,Best Smart Home-Enabled,link,https://www.homedepot.com/p/remo-RemoBell-S-Sm...


In [7]:

spruce_search_bestlinks('Doorbell Cameras')[0]

'https://www.thespruce.com/best-doorbell-cameras-4147384'

In [8]:
diction = spruce_search_top_pics('https://www.thespruce.com/best-doorbell-cameras-4147384')

The 8 Best Doorbell Cameras of 2020


In [9]:
get_table_spruce("clocks")
#Adding a comment 


The 8 Best Wall Clocks of 2020


,Name,Cat,Source,Link
0,La Crosse Technology 14-Inch Atomic Wall Clock,Best Overall,amazon,https://www.amazon.com/Crosse-Technology-WT-31...
1,Bernhard Products 10-Inch Black Wall Clock,Best Budget,amazon,https://www.amazon.com/Bernhard-Products-Ticki...
2,Infinity Instruments Orange Retro Wall Clock,Best for the Office,amazon,https://www.amazon.com/Infinity-Instruments-Or...
3,Seiko Wall Pendulum Schoolhouse Clock,Best for the Kitchen,amazon,https://www.amazon.com/Seiko-Pendulum-Schoolho...
4,Umbra Ribbon Wall Clock,Best for the Dining Room,amazon,https://www.amazon.com/Umbra-Ribbon-Clock-Stai...
5,"Latitude Run Soltis 16"" Wall Clock at Wayfair",Best for Living Room,link,https://www.wayfair.com/decor-pillows/pdp/lati...
6,"Laurel Foundry Modern Farmhouse 30"" Wall Clock...",Best for Bathroom,link,https://www.wayfair.com/decor-pillows/pdp/over...
